#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [ ]:
import pandas as pd

#Input Lists Example (No Constraint Lists)
Below, I review a problem that writes the input parameters in a way that is scalable for a large number of decision variables utilizing lists.


In [ ]:
#Minimize 5*x1 + 2*x2 + 8*x3 + 7x4
#such that
#          x2 + 2*x3 + 3*x4  >= 4
# 4*x1 + 2*x2 - 3*x3 + 9*x4  >= 7
# 2*x1 - 1*x2 + 2*x3 + 1*x4  >= 4
# x1, x2, x3, x4 >= 0
# x1, x2, x3, x4 <= 20


In [ ]:
#utilize lists to write inputs in a more scalable way
num_dvs = 4
obj_coef = [5, 2, 8, 7]
c1_coef = [0, 1, 2, 3]
c2_coef = [4, 2, -3, 9]
c3_coef = [2, -1, 2, 1]
c1_rhs = 4
c2_rhs = 7
c3_rhs = 4

In [ ]:
#initialize a "Concrete Model"
model = ConcreteModel()

#initialize DVs
model.x = Var(range(num_dvs), bounds = (0,20))

#define the objective
model.Objective = Objective(expr = sum(obj_coef[i]*model.x[i] for i in range(num_dvs)), sense = minimize)

#specify the constraints
model.Constraint1 = Constraint(expr = sum(c1_coef[i]*model.x[i] for i in range(num_dvs)) >= c1_rhs)
model.Constraint2 = Constraint(expr = sum(c2_coef[i]*model.x[i] for i in range(num_dvs)) >= c2_rhs)
model.Constraint3 = Constraint(expr = sum(c3_coef[i]*model.x[i] for i in range(num_dvs)) >= c3_rhs)

#(Optional) You can use model.pprint() to see what you've done so far
model.pprint()

1 Var Declarations
    x : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :    20 : False :  True :  Reals
          1 :     0 :  None :    20 : False :  True :  Reals
          2 :     0 :  None :    20 : False :  True :  Reals
          3 :     0 :  None :    20 : False :  True :  Reals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 5*x[0] + 2*x[1] + 8*x[2] + 7*x[3]

3 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                            : Upper : Active
        None :   4.0 : 0*x[0] + x[1] + 2*x[2] + 3*x[3] :  +Inf :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body                              : Upper : Active
        None :   7.0 : 4*x[0] + 2*x[1] - 3*x[2] + 9*x[3] :  +Inf :   True
    Constraint3 : Size=1, Index=N

In [ ]:
#solve model
opt = SolverFactory('cbc')

results = opt.solve(model, tee = True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmpd0gtbgow.pyomo.lp -stat=1 -solve -solu /tmp/tmpd0gtbgow.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 3 (0) rows, 4 (0) columns and 11 (0) elements
Statistics for presolved model


Problem has 3 rows, 4 columns (4 with objective) and 11 elements
Column breakdown:
0 of type 0.0->inf, 4 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
3 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 3 (0) rows, 4 (0) columns and 11 (0) elements
0  Obj 0 Primal inf 4.1111108 (3)
2  Obj 16
Optimal - 

In [ ]:
#print relevant values
for i in range(num_dvs):
  print(f"x{i} = {model.x[i]()}")
print("obj* = ", model.Objective())

x0 = 1.3333333
x1 = 0.0
x2 = 0.0
x3 = 1.3333333
obj* =  15.9999996


##Example with Input Lists and Constraint Lists
Below, I reprogram the exact same problem as above, but I organize the coefficients into lists and use ConstraintLists() in Pyomo together with a for loop. Writing optimization models in this way makes them scalable to large numbers of constraints.


In [ ]:
#Minimize 5*x1 + 2*x2 + 8*x3 + 7x4
#such that
#          x2 + 2*x3 + 3*x4  >= 4
# 4*x1 + 2*x2 - 3*x3 + 9*x4  >= 7
# 2*x1 - 1*x2 + 2*x3 + 1*x4  >= 4
# x1, x2, x3, x4 >= 0
# x1, x2, x3, x4 <= 20


In [ ]:
#utilize lists to write things in a more scalable way
num_dvs = 4 #indexed by i
num_constraints = 3 #indexed by j
obj_coef = [5, 2, 8, 7] #so obj_coef[i] is for dv i
c_coef = [[0, 1, 2, 3],[4, 2, -3, 9],[2, -1, 2, 1]] #so c_coef[j][i] is the coefficient for constraint j dv i
c_rhs = [4, 7, 4] #so c_rhs[j] is for constraint j


In [ ]:
#initialize a "Concrete Model"
model2 = ConcreteModel()

#initialize DVs
model2.x = Var(range(num_dvs), bounds = (0,20))

#define the objective
model2.Objective = Objective(expr = sum(obj_coef[i]*model2.x[i] for i in range(num_dvs)), sense = minimize)

#specify the constraints
model2.Constraints = ConstraintList()
for j in range(num_constraints):
  model2.Constraints.add(expr = sum(c_coef[j][i]*model2.x[i] for i in range(num_dvs)) >= c_rhs[j])

#(Optional) You can use model.pprint() to see what you've done so far
model2.pprint()

1 Var Declarations
    x : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :    20 : False :  True :  Reals
          1 :     0 :  None :    20 : False :  True :  Reals
          2 :     0 :  None :    20 : False :  True :  Reals
          3 :     0 :  None :    20 : False :  True :  Reals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 5*x[0] + 2*x[1] + 8*x[2] + 7*x[3]

1 Constraint Declarations
    Constraints : Size=3, Index={1, 2, 3}, Active=True
        Key : Lower : Body                              : Upper : Active
          1 :   4.0 :   0*x[0] + x[1] + 2*x[2] + 3*x[3] :  +Inf :   True
          2 :   7.0 : 4*x[0] + 2*x[1] - 3*x[2] + 9*x[3] :  +Inf :   True
          3 :   4.0 :     2*x[0] - x[1] + 2*x[2] + x[3] :  +Inf :   True

3 Declarations: x Objective Constraints


In [ ]:
#solve model
opt = SolverFactory('cbc')

results = opt.solve(model2, tee = True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmphwj37yiy.pyomo.lp -stat=1 -solve -solu /tmp/tmphwj37yiy.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 3 (0) rows, 4 (0) columns and 11 (0) elements
Statistics for presolved model


Problem has 3 rows, 4 columns (4 with objective) and 11 elements
Column breakdown:
0 of type 0.0->inf, 4 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
3 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 3 (0) rows, 4 (0) columns and 11 (0) elements
0  Obj 0 Primal inf 4.1111108 (3)
2  Obj 16
Optimal - 

In [ ]:
#print relevant values
for i in range(num_dvs):
  print(f"x{i} = {model2.x[i]()}")
print("obj* = ", model2.Objective())

x0 = 1.3333333
x1 = 0.0
x2 = 0.0
x3 = 1.3333333
obj* =  15.9999996
